In [1]:
# Import dependencies
import requests
from pprint import pprint
import pandas as pd
from datetime import datetime as dt
import sqlalchemy

### Extract the Data 

Make an API call to the database to fetch the data for February 2023.

In [2]:
# Define the URL for the database
URL = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?"

In [3]:
# Define the queries to make a call for only the data for February 2023
limit = "&$limit=10000"
query = "&$where=crash_date >= '2023-02-01T00:00:00.000' AND crash_date < '2023-03-01T00:00:00.000'"
query_url = URL+limit+query
response = requests.get(query_url)
data = response.json()

In [4]:
# Check data type
type(data)

list

In [5]:
# Check the limit was enough to extract all data points
len(data)

6828

In [6]:
## Pretty print a random data point
pprint(data[8])

{'borough': 'MANHATTAN',
 'collision_id': '4602575',
 'contributing_factor_vehicle_1': 'Driver Inattention/Distraction',
 'crash_date': '2023-02-01T00:00:00.000',
 'crash_time': '0:00',
 'latitude': '40.79748',
 'location': {'human_address': '{"address": "", "city": "", "state": "", '
                               '"zip": ""}',
              'latitude': '40.79748',
              'longitude': '-73.94879'},
 'longitude': '-73.94879',
 'number_of_cyclist_injured': '0',
 'number_of_cyclist_killed': '0',
 'number_of_motorist_injured': '0',
 'number_of_motorist_killed': '0',
 'number_of_pedestrians_injured': '1',
 'number_of_pedestrians_killed': '0',
 'number_of_persons_injured': '1',
 'number_of_persons_killed': '0',
 'off_street_name': '5 AVENUE',
 'on_street_name': 'WEST 111 STREET',
 'vehicle_type_code1': 'Sedan',
 'zip_code': '10026'}


In [7]:
# Convert the data to a Pandas dataframe
df = pd.DataFrame(data)
df.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,number_of_persons_injured,...,vehicle_type_code1,vehicle_type_code2,cross_street_name,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,vehicle_type_code_3,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2023-02-01T00:00:00.000,18:55,STATEN ISLAND,10301,40.639652,-74.07551,"{'latitude': '40.639652', 'longitude': '-74.07...",BAY STREET,SLOSSON TERRACE,1,...,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-01T00:00:00.000,20:23,BROOKLYN,11208,40.669655,-73.87589,"{'latitude': '40.669655', 'longitude': '-73.87...",NaN,NaN,0,...,Sedan,Fire Truck,370 MILFORD STREET,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-01T00:00:00.000,16:15,BROOKLYN,11233,40.676342,-73.910934,"{'latitude': '40.676342', 'longitude': '-73.91...",ATLANTIC AVENUE,ROCKAWAY AVENUE,1,...,Carry All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-01T00:00:00.000,14:40,BROOKLYN,11225,40.670784,-73.96786,"{'latitude': '40.670784', 'longitude': '-73.96...",NaN,NaN,1,...,Sedan,Sedan,418 FLATBUSH AVENUE,Unspecified,Unspecified,Unspecified,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN
4,2023-02-01T00:00:00.000,0:45,BROOKLYN,11212,40.659367,-73.90132,"{'latitude': '40.659367', 'longitude': '-73.90...",NaN,NaN,1,...,Sedan,Sedan,606 POWELL STREET,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN


## Transform

Transform the data set to required data types, and add and drop columns as necessary.

In [8]:
# Check the data types for the different columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6828 entries, 0 to 6827
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crash_date                     6828 non-null   object
 1   crash_time                     6828 non-null   object
 2   borough                        4484 non-null   object
 3   zip_code                       4483 non-null   object
 4   latitude                       6360 non-null   object
 5   longitude                      6360 non-null   object
 6   location                       6360 non-null   object
 7   on_street_name                 4994 non-null   object
 8   off_street_name                3168 non-null   object
 9   number_of_persons_injured      6828 non-null   object
 10  number_of_persons_killed       6828 non-null   object
 11  number_of_pedestrians_injured  6828 non-null   object
 12  number_of_pedestrians_killed   6828 non-null   object
 13  num

In [9]:
# Drop any rows which do not have any vehicle associated with them
df_dropped = df[df['vehicle_type_code1'].notna()]
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6725 entries, 0 to 6827
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crash_date                     6725 non-null   object
 1   crash_time                     6725 non-null   object
 2   borough                        4401 non-null   object
 3   zip_code                       4400 non-null   object
 4   latitude                       6258 non-null   object
 5   longitude                      6258 non-null   object
 6   location                       6258 non-null   object
 7   on_street_name                 4911 non-null   object
 8   off_street_name                3099 non-null   object
 9   number_of_persons_injured      6725 non-null   object
 10  number_of_persons_killed       6725 non-null   object
 11  number_of_pedestrians_injured  6725 non-null   object
 12  number_of_pedestrians_killed   6725 non-null   object
 13  num

In [10]:
# Convert the 'crash_date' column to datetime data type
df_dropped['crash_date']= pd.to_datetime(df_dropped['crash_date'])
df_dropped.head()

C:\Users\Brittni\AppData\Local\Temp\ipykernel_15980\2598217026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropped['crash_date']= pd.to_datetime(df_dropped['crash_date'])


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,number_of_persons_injured,...,vehicle_type_code1,vehicle_type_code2,cross_street_name,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,vehicle_type_code_3,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2023-02-01,18:55,STATEN ISLAND,10301,40.639652,-74.07551,"{'latitude': '40.639652', 'longitude': '-74.07...",BAY STREET,SLOSSON TERRACE,1,...,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-01,20:23,BROOKLYN,11208,40.669655,-73.87589,"{'latitude': '40.669655', 'longitude': '-73.87...",NaN,NaN,0,...,Sedan,Fire Truck,370 MILFORD STREET,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-01,16:15,BROOKLYN,11233,40.676342,-73.910934,"{'latitude': '40.676342', 'longitude': '-73.91...",ATLANTIC AVENUE,ROCKAWAY AVENUE,1,...,Carry All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-01,14:40,BROOKLYN,11225,40.670784,-73.96786,"{'latitude': '40.670784', 'longitude': '-73.96...",NaN,NaN,1,...,Sedan,Sedan,418 FLATBUSH AVENUE,Unspecified,Unspecified,Unspecified,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN
4,2023-02-01,0:45,BROOKLYN,11212,40.659367,-73.90132,"{'latitude': '40.659367', 'longitude': '-73.90...",NaN,NaN,1,...,Sedan,Sedan,606 POWELL STREET,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Convert the 'crash_time' column to datetime data type and keep only the hour
df_dropped['crash_time'] = pd.to_datetime(df_dropped['crash_time'], format='%H:%M').dt.hour
df_dropped.head()

C:\Users\Brittni\AppData\Local\Temp\ipykernel_15980\3243245947.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropped['crash_time'] = pd.to_datetime(df_dropped['crash_time'], format='%H:%M').dt.hour


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,number_of_persons_injured,...,vehicle_type_code1,vehicle_type_code2,cross_street_name,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,vehicle_type_code_3,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2023-02-01,18,STATEN ISLAND,10301,40.639652,-74.07551,"{'latitude': '40.639652', 'longitude': '-74.07...",BAY STREET,SLOSSON TERRACE,1,...,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-01,20,BROOKLYN,11208,40.669655,-73.87589,"{'latitude': '40.669655', 'longitude': '-73.87...",NaN,NaN,0,...,Sedan,Fire Truck,370 MILFORD STREET,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-01,16,BROOKLYN,11233,40.676342,-73.910934,"{'latitude': '40.676342', 'longitude': '-73.91...",ATLANTIC AVENUE,ROCKAWAY AVENUE,1,...,Carry All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-01,14,BROOKLYN,11225,40.670784,-73.96786,"{'latitude': '40.670784', 'longitude': '-73.96...",NaN,NaN,1,...,Sedan,Sedan,418 FLATBUSH AVENUE,Unspecified,Unspecified,Unspecified,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN
4,2023-02-01,0,BROOKLYN,11212,40.659367,-73.90132,"{'latitude': '40.659367', 'longitude': '-73.90...",NaN,NaN,1,...,Sedan,Sedan,606 POWELL STREET,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Convert the various numbers of injuries/deaths columns to integer types
columns_to_int = ['number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
                  'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured', 'number_of_motorist_killed']
df_dropped[columns_to_int] = df_dropped[columns_to_int].astype(int)
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6725 entries, 0 to 6827
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_date                     6725 non-null   datetime64[ns]
 1   crash_time                     6725 non-null   int64         
 2   borough                        4401 non-null   object        
 3   zip_code                       4400 non-null   object        
 4   latitude                       6258 non-null   object        
 5   longitude                      6258 non-null   object        
 6   location                       6258 non-null   object        
 7   on_street_name                 4911 non-null   object        
 8   off_street_name                3099 non-null   object        
 9   number_of_persons_injured      6725 non-null   int32         
 10  number_of_persons_killed       6725 non-null   int32         
 11  number_of_pedestr

C:\Users\Brittni\AppData\Local\Temp\ipykernel_15980\636782480.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropped[columns_to_int] = df_dropped[columns_to_int].astype(int)


In [13]:
# Convert lat and lon columns to float data type
df_dropped[['latitude','longitude']] = df_dropped[['latitude','longitude']].astype(float)
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6725 entries, 0 to 6827
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_date                     6725 non-null   datetime64[ns]
 1   crash_time                     6725 non-null   int64         
 2   borough                        4401 non-null   object        
 3   zip_code                       4400 non-null   object        
 4   latitude                       6258 non-null   float64       
 5   longitude                      6258 non-null   float64       
 6   location                       6258 non-null   object        
 7   on_street_name                 4911 non-null   object        
 8   off_street_name                3099 non-null   object        
 9   number_of_persons_injured      6725 non-null   int32         
 10  number_of_persons_killed       6725 non-null   int32         
 11  number_of_pedestr

C:\Users\Brittni\AppData\Local\Temp\ipykernel_15980\999993061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropped[['latitude','longitude']] = df_dropped[['latitude','longitude']].astype(float)


In [14]:
# Bin the rows based on the hour when the incident happened
df_binned = df_dropped
bins = [0,4,8,12,16,20,24]
labels = ["12AM - 4AM","4AM - 8AM","8AM - 12PM","12PM - 4PM","4PM - 8PM","8PM - 12AM"]
df_binned['period_of_day'] = pd.cut(x = df_binned['crash_time'], bins = bins, labels = labels)
df_binned.head()

C:\Users\Brittni\AppData\Local\Temp\ipykernel_15980\89123022.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_binned['period_of_day'] = pd.cut(x = df_binned['crash_time'], bins = bins, labels = labels)


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,number_of_persons_injured,...,vehicle_type_code2,cross_street_name,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,vehicle_type_code_3,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5,period_of_day
0,2023-02-01,18,STATEN ISLAND,10301,40.639652,-74.075510,"{'latitude': '40.639652', 'longitude': '-74.07...",BAY STREET,SLOSSON TERRACE,1,...,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4PM - 8PM
1,2023-02-01,20,BROOKLYN,11208,40.669655,-73.875890,"{'latitude': '40.669655', 'longitude': '-73.87...",NaN,NaN,0,...,Fire Truck,370 MILFORD STREET,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,4PM - 8PM
2,2023-02-01,16,BROOKLYN,11233,40.676342,-73.910934,"{'latitude': '40.676342', 'longitude': '-73.91...",ATLANTIC AVENUE,ROCKAWAY AVENUE,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12PM - 4PM
3,2023-02-01,14,BROOKLYN,11225,40.670784,-73.967860,"{'latitude': '40.670784', 'longitude': '-73.96...",NaN,NaN,1,...,Sedan,418 FLATBUSH AVENUE,Unspecified,Unspecified,Unspecified,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,12PM - 4PM
4,2023-02-01,0,BROOKLYN,11212,40.659367,-73.901320,"{'latitude': '40.659367', 'longitude': '-73.90...",NaN,NaN,1,...,Sedan,606 POWELL STREET,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Create a new database and keep only the columns needed
df_final = df_binned[['collision_id','crash_date','crash_time','borough','latitude','longitude','number_of_persons_injured', 'number_of_persons_killed', 
                      'number_of_pedestrians_injured', 'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 
                      'number_of_motorist_injured', 'number_of_motorist_killed', 'vehicle_type_code1', 'vehicle_type_code2', 'vehicle_type_code_3',
                      'vehicle_type_code_4','vehicle_type_code_5']]
df_final.head()

,collision_id,crash_date,crash_time,borough,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,4603486,2023-02-01,18,STATEN ISLAND,40.639652,-74.075510,1,0,0,0,1,0,0,0,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
1,4602459,2023-02-01,20,BROOKLYN,40.669655,-73.875890,0,0,0,0,0,0,0,0,Sedan,Fire Truck,NaN,NaN,NaN
2,4602515,2023-02-01,16,BROOKLYN,40.676342,-73.910934,1,0,1,0,0,0,0,0,Carry All,NaN,NaN,NaN,NaN
3,4603107,2023-02-01,14,BROOKLYN,40.670784,-73.967860,1,0,0,0,0,0,1,0,Sedan,Sedan,Sedan,Station Wagon/Sport Utility Vehicle,NaN
4,4602182,2023-02-01,0,BROOKLYN,40.659367,-73.901320,1,0,1,0,0,0,0,0,Sedan,Sedan,NaN,NaN,NaN


## Load

Load the pandas dataframe to a SQlite database.

In [129]:
# Create an engine to access the sqlite database file
engine=sqlalchemy.create_engine('sqlite:///data/db.sqlite')

In [130]:
# Convert the dataframe to a "data" table in the db.sqlite database
df_final.to_sql('data', if_exists='replace', index=False, con=engine)